In [143]:
from datetime import timedelta
import torch
from torch.autograd import Variable
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models,transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import TensorDataset, DataLoader,random_split
from __future__ import print_function
import numpy as np 
import pandas as pd
import os
import time
import random
import matplotlib.pyplot as plt 
from torch.autograd import Function
from collections import OrderedDict
import torch.nn as nn
import math
import torchvision.models as models
import pickle
import cv2
import os
import sys
import glob
import collections
import pandas as pd
import wandb
from sklearn.model_selection import KFold
# from torchsummary import summary
import glob
from sklearn.manifold import TSNE
import seaborn as sns
import collections
from transformers import BertConfig, BertModel


import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

SAVING_FRAMES_PER_SECOND = 30

In [8]:
VIDEO_FOLDER = "/media/data_cifs/nih/files_to_send/"
ACTION_FOLDER = "/media/data_cifs/nih/inference/results_csv/"

In [9]:
def cohort_search(cohort, search_root, ret=False):
    cohort = str(cohort).upper()
    all_folders = [folder for folder in os.listdir(search_root) if folder.startswith("FC-"+cohort)]
    all_folders.sort()
    all_folder_list = "\n".join(all_folders)
    if ret:
        return all_folders
    else:
        print(all_folder_list)

In [10]:
def find_mouse_videos(mouse_id, cohort, ret=False):
    
    search_for = "v"
    
    if search_for == "videos":
        print("Looking for videos..")
        file_extension = ".mp4" 
        ROOT_FOLDER = VIDEO_FOLDER
    else:
        print("Looking for actions...")
        file_extension = ".csv"
        ROOT_FOLDER = ACTION_FOLDER
    
    mouse_id = str(mouse_id)
    cohort_folders = cohort_search(cohort, ROOT_FOLDER, ret=True)
    
    preexposure_folders = [folder for folder in cohort_folders if "exp" in folder]
    postcond_folders = [folder for folder in cohort_folders if "cond" in folder]
    postext_folders = [folder for folder in cohort_folders if "xt" in folder]
    postret_folders = [folder for folder in cohort_folders if "tre" in folder]
    
    preexposure_videos = []
    postcond_videos = []
    postext_videos = []
    postret_videos = []
    
    for folder in preexposure_folders:
        preexposure_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    preexposure_videos.sort()
    
    for folder in postcond_folders:
        postcond_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postcond_videos.sort()
    
    for folder in postext_folders:
        postext_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postext_videos.sort()
    
    for folder in postret_folders:
        postret_videos += [video+'/'+folder for video in os.listdir(ROOT_FOLDER+folder) if (mouse_id in video and video.endswith(file_extension) and os.stat(ROOT_FOLDER+folder+'/'+video).st_size)]
    postret_videos.sort()
    
    if ret:
        return preexposure_videos, postcond_videos
    else:
        print("\nPREEXPOSURE:")
        print("\n".join(preexposure_videos))
        print("\nPOSTCONDITIONING:")
        print("\n".join(postcond_videos))
        print("\nPOSTEXTINCTION:")
        print("\n".join(postext_videos))
        print("\nPOSTRETRIEVAL:")
        print("\n".join(postret_videos))

In [11]:
BEH_LABELS = ['drink', 'eat', 'groom', 'hang', 'sniff', 'rear', 'rest', 'walk', 'nibble']
action_colours = ['tab:blue', 'tab:orange', 'limegreen', 'deepskyblue', 'tab:pink', 'tab:purple', 'tab:brown', 'red', 'black']
len(action_colours) == len(BEH_LABELS)

True

In [279]:
# READING THE CSV: postcond2_actions[0].iloc[2][1]
# Loading actions
# 17202338:
preexposure_actions_17202338_a = []
postcond_actions_17202338_a = []
postext_actions_17202338_a = []
postret_actions_17202338_a = []

preexposure_video_list_17202338_a = [
"video_2018Y_08M_10D_23h_56m_27s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_00h_56m_27s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_01h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_02h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_03h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_04h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_05h_56m_28s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_06h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_07h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_09h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_10h_56m_29s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_11h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_12h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_13h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_14h_56m_30s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_15h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_16h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_17h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_18h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_19h_56m_31s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_20h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_21h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_22h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_11D_23h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_00h_56m_32s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_01h_56m_33s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_02h_56m_33s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_03h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_04h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_05h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_06h_56m_34s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
"video_2018Y_08M_12D_07h_56m_35s_cam_17202338-0000.csv/FC-A_08-09-18_preexposure",
]


for action_list in preexposure_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    preexposure_actions_17202338_a.append(actions)

print("Preexposure Files:", len(preexposure_actions_17202338_a))

postcond_video_list_17202338_a = [
"video_2018Y_08M_13D_23h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_00h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_01h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_02h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_03h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_04h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_05h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_06h_21m_05s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_07h_21m_06s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_09h_28m_54s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_10h_28m_55s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_11h_28m_55s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_12h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_13h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_14h_28m_56s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_15h_28m_57s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_16h_28m_57s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_17h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_18h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_19h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_20h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_21h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_22h_28m_58s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_14D_23h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_00h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_01h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_02h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_03h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_04h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_05h_28m_59s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_06h_29m_00s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
"video_2018Y_08M_15D_07h_29m_00s_cam_17202338-0000.csv/FC-A_08-10-18_postcond",
]

for action_list in postcond_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postcond_actions_17202338_a.append(actions)
print("Postcond Files:", len(postcond_actions_17202338_a))
    
postext_video_list_17202338_a = [
"video_2018Y_08M_15D_23h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_00h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_01h_58m_47s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_02h_58m_48s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
"video_2018Y_08M_16D_03h_58m_49s_cam_17202338-0000.csv/FC-A_08-15-18_postext",
]

for action_list in postext_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postext_actions_17202338_a.append(actions)

print("Postext Files:", len(postext_actions_17202338_a))

postret_video_list_17202338_a = [
"video_2018Y_08M_20D_23h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_00h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_01h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_02h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
"video_2018Y_08M_21D_03h_54m_37s_cam_17202338-0000.csv/FC-A_08-20-18_postret",
]

for action_list in postret_video_list_17202338_a:
    folder_name = action_list.split('/')[-1]
    action_file = action_list.split('/')[0]
    csv_name = ACTION_FOLDER+folder_name+"/"+action_file
    actions = pd.read_csv(csv_name)
    print("Reading CSV:", csv_name)
    postret_actions_17202338_a.append(actions)

print("Postret Files:", len(postret_actions_17202338_a))

Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_10D_23h_56m_27s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_00h_56m_27s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_01h_56m_28s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_02h_56m_28s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_03h_56m_28s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_04h_56m_28s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-09-18_preexposure/video_2018Y_08M_11D_05h_56m_28s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/resu

Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-10-18_postcond/video_2018Y_08M_15D_07h_29m_00s_cam_17202338-0000.csv
Postcond Files: 32
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-15-18_postext/video_2018Y_08M_15D_23h_58m_47s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-15-18_postext/video_2018Y_08M_16D_00h_58m_47s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-15-18_postext/video_2018Y_08M_16D_01h_58m_47s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-15-18_postext/video_2018Y_08M_16D_02h_58m_48s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-15-18_postext/video_2018Y_08M_16D_03h_58m_49s_cam_17202338-0000.csv
Postext Files: 5
Reading CSV: /media/data_cifs/nih/inference/results_csv/FC-A_08-20-18_postret/video_2018Y_08M_20D_23h_54m_37s_cam_17202338-0000.csv
Reading CSV: /media/data_cifs/nih/infer

In [280]:
all_preexposure_actions_17202338_a = []
for action_set in preexposure_actions_17202338_a:
    for idx in range(len(action_set)):
        all_preexposure_actions_17202338_a.append(action_set.iloc[idx][1])
print("Preexposure:", len(all_preexposure_actions_17202338_a))

all_postcond_actions_17202338_a = []
for action_set in postcond_actions_17202338_a:
    for idx in range(len(action_set)):
        all_postcond_actions_17202338_a.append(action_set.iloc[idx][1])
print("Postcond:", len(all_postcond_actions_17202338_a))

# all_postext_actions_17202338_a = []
# for action_set in postext_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_postext_actions_17202338_a.append(action_set.iloc[idx][1])
# print("Postext:", len(all_postext_actions_17202338_a))

# all_postret_actions_17202338_a = []
# for action_set in postret_actions_17202338_a:
#     for idx in range(len(action_set)):
#         all_postret_actions_17202338_a.append(action_set.iloc[idx][1])
# print("Postret:", len(all_postret_actions_17202338_a))

Preexposure: 3450848
Postcond: 3450848


In [281]:
time_counter = {'0-5':0,
    '5-15':0,
    '15-45':0,
    '45-90':0,
    '90-450':0,
    '450-1800':0,
    '1800-4500':0,
    'over4500':0,}

def get_repeat_vocab(repeat_number):
    # > '0-5'
    # > '5-15'
    # > '15-45'
    # > '45-90'
    # > '90-450'
    # > '450-1800'
    # > '1800-4500'
    # > 'over4500'
    if repeat_number > 0 and repeat_number <= 5:
        box = "0-5"
    elif repeat_number > 5 and repeat_number <= 15:
        box = "5-15"
    elif repeat_number > 15 and repeat_number <= 45:
        box = "15-45"
    elif repeat_number > 45 and repeat_number <= 90:
        box = "45-90"
    elif repeat_number > 90 and repeat_number <= 450:
        box = "90-450"
    elif repeat_number > 450 and repeat_number <= 1800:
        box = "450-1800"
    elif repeat_number > 1800 and repeat_number <= 4500:
        box = "1800-4500"
    elif repeat_number > 4500:
        box = "over4500"
    else:
        raise ValueError('A very specific bad thing happened.')
    time_counter[box] += 1
    return box

In [316]:
def generate_language_from_labels(action_list, class_label):
    SENTENCE_LENGTH = 30 # in terms of actions
    
    start_index = 0
    action_length = len(action_list)
    
    language = []
    label = []
    sentence = ""
    
    current_sentence_length = 0
    while start_index < action_length:
        
        current_action = action_list[start_index]
        current_action_label = BEH_LABELS[current_action]
        
        next_action_idx = start_index + 1
        while next_action_idx < action_length and current_action == action_list[next_action_idx]:
            next_action_idx  += 1
        repeats = next_action_idx - start_index
        
        if current_sentence_length == 0:
            sentence = ""
            
        current_sentence_length += repeats
        start_index = next_action_idx
        
        sentence += str(current_action_label)+" "+get_repeat_vocab(repeats)+" "
        
        if current_sentence_length >= SENTENCE_LENGTH:
            language.append(sentence)
            label.append(class_label)
            current_sentence_length = 0
            
    if sentence != "":
        language.append(sentence)
        label.append(class_label)
    return language, label

In [317]:
# print("Language Length:", len(generate_language_from_labels(all_preexposure_actions_17202338_a, 0)))
# print("Original Action Length:", len(all_preexposure_actions_17202338_a))

In [318]:
raw_text = []
labels = []

preexposure = generate_language_from_labels(all_preexposure_actions_17202338_a, 0)
postcond = generate_language_from_labels(all_postcond_actions_17202338_a, 1)

raw_text += preexposure[0]
labels += preexposure[1]

raw_text += postcond[0]
labels += postcond[1]

In [319]:
len(raw_text), len(labels), len(raw_text) == len(labels)

(44667, 44667, True)

In [320]:
print(time_counter)

{'0-5': 220053, '5-15': 94695, '15-45': 81021, '45-90': 29013, '90-450': 23271, '450-1800': 2988, '1800-4500': 1137, 'over4500': 687}


In [321]:
# Initializing a BERT bert-base-uncased style configuration
configuration = BertConfig(vocab_size = 20, hidden_size = 24, num_hidden_layers = 6,  num_attention_heads = 12, intermediate_size = 1024, hidden_act = 'gelu', hidden_dropout_prob = 0.1, attention_probs_dropout_prob = 0.1,  max_position_embeddings = 45, type_vocab_size = 2, initializer_range = 0.02, layer_norm_eps = 1e-12, pad_token_id = 0, position_embedding_type = 'absolute',  use_cache = True, classifier_dropout = None )

In [322]:
# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = BertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [323]:
# Define Vocabulary:
# > '0-5'
# > '5-15'
# > '15-45'
# > '45-90'
# > '90-450'
# > '450-1800'
# > '1800-4500'
# > 'over4500'
# > 'drink'
# > 'eat'
# > 'groom'
# > 'hang'
# > 'sniff'
# > 'rear'
# > 'rest'
# > 'walk'
# > 'eathand'
# > [CLS]
# > [MASK]
# > [SEP]

# TOTAL SIZE = 20

In [353]:
from tqdm import tqdm
from transformers import BertTokenizerFast

# repositor id for saving the tokenizer
tokenizer_id="bert-base-uncased-2022-habana"

# create a python generator to dynamically load the data
def batch_iterator(batch_size=100):
    for i in tqdm(range(0, len(raw_text), batch_size)):
        print("Batch:", raw_text[i : i + batch_size])
        yield raw_datasets[i : i + batch_size]

# create a tokenizer from existing one to re-use special tokens
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [352]:
# tokenizer = tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size = 20)
# tokenizer.save_pretrained("tokenizer")

100%|██████████████████████████████████████| 447/447 [00:00<00:00, 11205.00it/s]

Batch: ['eat 5-15 hang 45-90 ', 'eat 45-90 ', 'hang 0-5 eat 5-15 hang 45-90 ', 'eat 0-5 hang 45-90 ', 'eat 45-90 ', 'hang 45-90 ', 'eat 90-450 ', 'hang 0-5 eat 15-45 ', 'hang 0-5 eat 0-5 hang 15-45 ', 'eat 5-15 hang 5-15 eat 0-5 hang 0-5 eat 0-5 hang 0-5 eat 0-5 hang 0-5 eat 0-5 hang 0-5 ', 'eat 5-15 hang 0-5 eat 0-5 hang 5-15 eat 15-45 ', 'hang 0-5 eat 15-45 hang 45-90 ', 'eat 0-5 hang 90-450 ', 'eat 0-5 hang 0-5 eat 0-5 hang 0-5 eat 5-15 hang 45-90 ', 'eat 15-45 hang 45-90 ', 'eat 5-15 hang 5-15 eat 15-45 ', 'hang 450-1800 ', 'eat 15-45 hang 45-90 ', 'eat 0-5 hang 0-5 eat 5-15 hang 0-5 eat 0-5 hang 15-45 ', 'eat 45-90 ', 'hang 45-90 ', 'rear 0-5 eat 0-5 sniff 15-45 walk 5-15 ', 'sniff 0-5 rear 5-15 eat 15-45 groom 5-15 ', 'sniff 45-90 ', 'walk 15-45 sniff 0-5 walk 0-5 sniff 0-5 eat 45-90 ', 'hang 45-90 ', 'eat 0-5 hang 45-90 ', 'eat 0-5 rear 0-5 sniff 5-15 rear 0-5 eat 90-450 ', 'hang 90-450 ', 'eat 15-45 hang 5-15 eat 0-5 ', 'hang 0-5 eat 0-5 rear 0-5 eat 0-5 rear 0-5 eat 0-5 rear 0

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [355]:
def print_rand_sentence():
    '''
    Displays the tokens and respective IDs of a random text sample
    '''
    index = random.randint(0, len(raw_text)-1)
    table = np.array([tokenizer.tokenize(raw_text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text[index]))]).T
    print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒══════════╤═════════════╕
│ Tokens   │   Token IDs │
╞══════════╪═════════════╡
│ walk     │        3328 │
├──────────┼─────────────┤
│ 15       │        2321 │
├──────────┼─────────────┤
│ -        │        1011 │
├──────────┼─────────────┤
│ 45       │        3429 │
├──────────┼─────────────┤
│ rear     │        4373 │
├──────────┼─────────────┤
│ 0        │        1014 │
├──────────┼─────────────┤
│ -        │        1011 │
├──────────┼─────────────┤
│ 5        │        1019 │
├──────────┼─────────────┤
│ eat      │        4521 │
├──────────┼─────────────┤
│ 45       │        3429 │
├──────────┼─────────────┤
│ -        │        1011 │
├──────────┼─────────────┤
│ 90       │        3938 │
╘══════════╧═════════════╛


In [356]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
    '''
    Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
    '''
    return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 60,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in raw_text:
    encoding_dict = preprocessing(sample, tokenizer)
    token_id.append(encoding_dict['input_ids']) 
    attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/tmp/ipykernel_3608321/3039929160.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [357]:
raw_text[0]

'eat 5-15 hang 45-90 '

In [358]:
len(token_id)

44667

In [359]:
labels

tensor([0, 0, 0,  ..., 1, 1, 1])

In [360]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)



# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size)

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size)

In [361]:
def b_tp(preds, labels):
    '''Returns True Positives (TP): count of correct predictions of actual class 1'''
    return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
    '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
    return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
    '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
    return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
    '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
    return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
    '''
    Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
    '''
    preds = np.argmax(preds, axis = 1).flatten()
    labels = labels.flatten()
    tp = b_tp(preds, labels)
    tn = b_tn(preds, labels)
    fp = b_fp(preds, labels)
    fn = b_fn(preds, labels)
    b_accuracy = (tp + tn) / len(labels)
    b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
    b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
    b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
    return b_accuracy, b_precision, b_recall, b_specificity

In [368]:
# Load the BertForSequenceClassificationjkk model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
# model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [369]:
device = 'cpu'# torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 4

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  25%|████████▌                         | 1/4 [36:53<1:50:39, 2213.06s/it]


	 - Train loss: 0.6961
	 - Validation Accuracy: 0.4983
	 - Validation Precision: 0.4983
	 - Validation Recall: 1.0000
	 - Validation Specificity: 0.0000



Epoch:  50%|████████████████                | 2/4 [1:13:41<1:13:40, 2210.15s/it]


	 - Train loss: 0.6949
	 - Validation Accuracy: 0.5017
	 - Validation Precision: NaN
	 - Validation Recall: 0.0000
	 - Validation Specificity: 1.0000



Epoch:  75%|█████████████████████████▌        | 3/4 [1:50:32<36:50, 2210.79s/it]


	 - Train loss: 0.6946
	 - Validation Accuracy: 0.4983
	 - Validation Precision: 0.4983
	 - Validation Recall: 1.0000
	 - Validation Specificity: 0.0000



Epoch: 100%|██████████████████████████████████| 4/4 [2:27:19<00:00, 2209.80s/it]


	 - Train loss: 0.6943
	 - Validation Accuracy: 0.5017
	 - Validation Precision: NaN
	 - Validation Recall: 0.0000
	 - Validation Specificity: 1.0000



In [ ]:
start = 0
end = 27.3488

starty